## Importing, time stepping, and grid spacing

In [1]:
import numpy as np
import climlab # package used for calculating orbital parameters overtime
from climlab.solar.orbital import OrbitalTable
from climlab.solar.insolation import daily_insolation
import matplotlib.pyplot as plt
from scipy.sparse import diags


#timesteping 
t_i = 0
t_f = -5000 #run model for the past 700,000 years 
d_t = 0.01  #use time step of 100 years
t = np.arange(t_f, t_i+1, d_t)
t_kyr = (t/(1000)) # years in kyr, needed to calculate orbital parameters

#grid spacing
x_i = 0 # corresponds to 74 degrees N latitute (northern boundary) 
x_f = 4889000 # corresponds to 30 degrees N latitude (southern boundary)
x = np.linspace(x_i, x_f, 40)

d_x = x_f/(len(x)-1)

Tokenization took: 2.30 ms
Type conversion took: 1.34 ms
Parser memory cleanup took: 0.00 ms


### Boundary and Initial Conditions
For the initial height of the ice sheet we assume a rectangular cross section with a height of 920.43 meters. <br>
The boundary conditions for the southern boundary is that the height is zero (Dirichlet) <br>
For the northern boundary there is a no flux boundary condition


In [2]:
# Parameters
A = 5.77e-4 #meters^-3 years^-1
alpha = 5
beta = 2

# Initialize glacier 
#want to the northern part of glacier to be 920 m and decrease moving south so the southern border is 0 m
h_ini = np.linspace(920, 0, len(x))
h_ini[1] = h_ini[0]


def D(H_vec):
    Diff = np.zeros(len(H_vec))
    for i in range(len(Diff)-1):
        if (i == 0):
            Diff[0] = 1e-3 #abs(H_vec[1] - H_vec[0])**beta * H_vec[0]**alpha / (2*d_x)**beta
        elif(i == len(Diff - 1) ):
            Diff[i] = abs(0 - H_vec[i-1])**beta * H_vec[i]**alpha / (2*d_x)**beta
        else:
            Diff[i] = abs(H_vec[i+1] - H_vec[i-1])**beta * H_vec[i]**alpha / (2*d_x)**beta

    return Diff


def Alpha(D_vec):
    Alpha_diag = np.zeros((len(D_vec))-1)
    for i in range(1, len(D_vec)-1):
        Alpha_diag[i-1] = d_t/(4*d_x**2) * ( -D_vec[i+1] + D_vec[i-1] + 4 * D_vec[i]) * A * 0.7**(beta+1)
    Alpha_diag[-1] = 0
    return Alpha_diag

def Beta(D_vec):
    Beta_diag = np.zeros(len(D_vec))
    
    for i in range(0, len(D_vec)-1):
        Beta_diag[i] = d_t * ( (-2 * D_vec[i]) / d_x**2 + 1/d_t) * A * 0.7**(beta+1)
    Beta_diag[0] = 1
    Beta_diag[-1] = 1
    return Beta_diag

def Gamma(D_vec):
    Gamma_diag = np.zeros(len(D_vec)-1)
    
    for i in range(1, len(D_vec)-1):
        Gamma_diag[i] = (d_t)/(4*d_x**2) * ( D_vec[i+1] - D_vec[i-1] + 4 * D_vec[i]) * A * 0.7**(beta+1)

    Gamma_diag[0] = -1
    return Gamma_diag


sols = np.zeros((len(x), len(t)))
D_ini = D(h_ini)
diffs = np.zeros((len(D_ini), len(t)))
sols[:, 0] = h_ini
diffs[:, 0] = D_ini
h = h_ini

for i in range(0,5): #len(t)-1):
    
    #Solve for diffusivity value at each grid point
    Diffusivity = D(h)
    diffs[:, i] = Diffusivity
    print(Diffusivity)
    print("")

    #calculate entries in P matrix
    lower_diag = Alpha(Diffusivity)
    center_diag = Beta(Diffusivity)
    upper_diag = Gamma(Diffusivity)

    #Build P matrix to solve
    diagonals = [lower_diag, center_diag, upper_diag]
    M = diags(diagonals, [-1, 0, 1]).toarray()
    h[0] = 0
    h[-1] = 0
    M[0,0] = 1
    M[0, 1] = -1
    M[-1, -1] = 1
    M[-1, -2] = 0
    M_inv = np.linalg.inv(M)
    h_new = np.matmul(M_inv, h)
    
    h = h_new
    #update solutions
    sols[:, i+1] = h

    #print(h)




[1.00000000e-03 2.33385955e+07 4.03592252e+07 1.56409881e+07
 1.35860092e+07 1.17529332e+07 1.01232831e+07 8.67963722e+06
 7.40559830e+06 6.28576232e+06 5.30568755e+06 4.45186349e+06
 3.71167979e+06 3.07339529e+06 2.52610690e+06 2.05971860e+06
 1.66491038e+06 1.33310724e+06 1.05644807e+06 8.27754709e+05
 6.40500815e+05 4.88780878e+05 3.67279162e+05 2.71238663e+05
 1.96430073e+05 1.39120734e+05 9.60436029e+04 6.43662062e+04
 4.16596011e+04 2.58673347e+04 1.52744024e+04 8.47620822e+03
 4.34752294e+03 2.01144394e+03 8.08354208e+02 2.64881507e+02
 6.28576232e+01 8.27754709e+00 2.58673347e-01 0.00000000e+00]

[1.00000000e-03 1.96254119e+33 3.39364452e+33 1.31510953e+33
 1.14234970e+33 9.88226628e+32 8.51200130e+32 7.29813493e+32
 6.22687976e+32 5.28528362e+32 4.46120341e+32 3.74327905e+32
 3.12090737e+32 2.58421598e+32 2.12403721e+32 1.73188198e+32
 1.39991373e+32 1.12092229e+32 8.88297786e+31 6.96004569e+31
 5.38555074e+31 4.10983747e+31 3.08820935e+31 2.28066787e+31
 1.65165155e+31 1.1697